1/ Importation des données TC et TITMC 

2/ Import et traitement du fichier des prénoms 

3/ Ajouter le genre des personnes 

4/ Codification des adresses 

5/ Ajout et traitement du code commune

6/ Retraiter et simplifier les qualités 

In [ ]:
# Lecture de la base inpi
import os
import s3fs
import pandas as pd
# Create filesystem object
S3_ENDPOINT_URL = "https://" + os.environ["AWS_S3_ENDPOINT"]
fs = s3fs.S3FileSystem(client_kwargs={'endpoint_url': S3_ENDPOINT_URL})
BUCKET = "radjerad/inpi"
FILE_KEY_S3 = "data_rep_pm_2017.csv"
FILE_PATH_S3 = BUCKET + "/" + FILE_KEY_S3

with fs.open(FILE_PATH_S3, mode="rb") as file_in:
    df_rep_pm = pd.read_csv(file_in, sep=";", encoding="utf-8")

In [ ]:
df_rep_pm.shape # 9 751 833

In [ ]:
df_rep_pm = df_rep_pm.rename({"code greffe": "code_greffe"}, axis=1)

In [ ]:
df_rep_pm = df_rep_pm.loc[:, ["code_greffe", "siren", "denomination", "forme_juridique_x", "qualité",
"nom_patronymique", "nom_usage", "prénoms", "type", "date_naissance", "ville_naissance", "adresse_ligne1", "adresse_ligne2",
"adresse_ligne3", "code_postal", "ville", "code_commune", "pays", "id_représentant"]]

In [ ]:
df_rep_pm = df_rep_pm.rename({"forme_juridique_x": "forme_juridique", "prénoms": "prenoms", "qualité": "qualite", "id_représentant": "id_representant"}, axis=1)

In [ ]:
df_rep_pm.head(10)

In [ ]:
df_rep_pm["qualite"].isna().sum() # 246 926

In [ ]:
# Traitement des cas où le nom et les prénoms sont manquants


In [ ]:
df_rep_pm.loc[df_rep_pm["prenoms"] == "-", "prenoms"] = ""

In [ ]:
df_rep_pm.loc[df_rep_pm["prenoms"] == "", :].shape # 4 448

In [ ]:
df_rep_pm.loc[(df_rep_pm["prenoms"] == "") & (df_rep_pm["type"] == "P.Physique"), :].shape # 4 448

In [ ]:
re1 = "(INDIVISION SUCCESSORALE DE M\.\s|\sM\.\s|^M\.\s|MME\.\s|MR\.\s|MELLE\.\s|MONSIEUR\.?\s|MADAME\.?\s|MLLE\.\s|REPRESENTEE\sPAR\sMONSIEUR\.?\s|REPRESENTEE\sPAR\s)"
df_rep_pm["top_civilite"] = df_rep_pm["nom_patronymique"].str.contains(re1, regex=True)

In [ ]:
df_rep_pm.loc[df_rep_pm["top_civilite"] == True, "nom_patronymique"].head()

In [ ]:
#df_rep_pm["contenu_civilite"] = df_rep_pm["nom_patronymique"].str.extract(re1)

In [ ]:
#df_rep_pm.loc[df_rep_pm["top_civilite"] == True, "contenu_civilite"].head()

In [ ]:
df_rep_pm["nom_patronymique2"] = df_rep_pm["nom_patronymique"]

In [ ]:
df_rep_pm["nom_patronymique2"] = df_rep_pm["nom_patronymique2"].str.replace(re1,"")

In [ ]:
from pandas import option_context
with option_context('display.max_colwidth', 400):
    print(df_rep_pm.loc[df_rep_pm["top_civilite"] == True, "nom_patronymique2"].head(10))

In [ ]:
# on repartit les noms et les prénoms dans les bonnes variables
# on a modifié un peu le code de l'insee pour enlever les representee par, ici aussi car on va essayer de prendre en compte les particules
particule = "(DU\s[A-Za-z]+|DE\sLA\s[A-Za-z]+|[A-Za-z]+\sDU\s[A-Za-z]+|[A-Za-z]+\sDE\sLA\s[A-Za-z]+|DE\s[A-Za-z]+|[A-Za-z]+\sDE\s[A-Za-z]+)"
#particule = "(DU\s|DE\sLA\s|\sDU\s|\sDE\sLA\s|DE\s|\sDE\s)"
df_rep_pm["top_particule"] = df_rep_pm["nom_patronymique"].str.contains(particule, regex=True)

In [ ]:
df_rep_pm.loc[(df_rep_pm["prenoms"] == "") | (df_rep_pm["prenoms"].isna()),:].shape # 1093022

In [ ]:
df_rep_pm.loc[(df_rep_pm["prenoms"] == ""),:].shape # 4448, mais ce sont tous des cas différents, donc on n'a pas envie de séparer nom et prénoms comme le fait l'insee

In [ ]:
df_rep_pm.loc[(df_rep_pm["top_particule"] == True) & (df_rep_pm["prenoms"] == ""), "nom_patronymique2"].head(10)

In [ ]:
df_rep_pm.loc[df_rep_pm["nom_usage"].isna(), :].head() # pas d'info parfois quand il y a l'info nom patronymique et prénoms 

In [ ]:
# nettoyage de la variable qualite 
df_rep_pm["qualite"].value_counts()

In [ ]:
df_rep_pm["qualite2"] = df_rep_pm["qualite"]

In [ ]:
df_rep_pm["qualite2"] = df_rep_pm["qualite2"].str.lower()
df_rep_pm["qualite2"] = df_rep_pm["qualite2"].str.strip()

In [ ]:
df_rep_pm["qualite2"].value_counts()

In [ ]:
# nettoyage de la dénomination
df_rep_pm["denomination2"] = df_rep_pm["denomination"]

In [ ]:
df_rep_pm["denomination2"] = df_rep_pm["denomination2"].str.lower()

In [ ]:
df_rep_pm["denomination2"].head(10)

In [ ]:
# codification des adresses
df_rep_pm.columns

In [ ]:
df_rep_pm["ville"].head(10)

In [ ]:
df_rep_pm.loc[df_rep_pm["ville"] == "", :].shape

In [ ]:
df_rep_pm.loc[(df_rep_pm["ville"].isna()) & (df_rep_pm["adresse_ligne3"].isna() == False), :].head(10) #90740

In [ ]:
df_rep_pm.loc[(df_rep_pm["ville"].isna()) & (df_rep_pm["adresse_ligne3"].isna() == False), "ville"] = \
    df_rep_pm.loc[(df_rep_pm["ville"].isna()) & (df_rep_pm["adresse_ligne3"].isna() == False), "adresse_ligne3"]

In [ ]:
df_rep_pm.columns

In [ ]:
df_rep_pm["commune"] = df_rep_pm["ville"]

In [ ]:
df_rep_pm["commune"] = df_rep_pm["commune"].str.lower()

In [ ]:
df_rep_pm["commune"].head(10)

In [ ]:
df_rep_pm["commune"] = (df_rep_pm["commune"].str.replace("à", "a")
                                            .str.replace("è", "e")
                                            .str.replace("é", "e")
                                            .str.replace("ç", "c")
                                            .str.replace("ù", "u")
                                            .str.replace("ô", "o"))

In [ ]:
df_rep_pm["commune"] = (df_rep_pm["commune"].str.replace("SAINT", "ST"))

In [ ]:
df_rep_pm["commune"] = df_rep_pm["commune"].str.replace(" ", "")

In [ ]:
df_rep_pm["commune"].value_counts()

In [ ]:
# Create filesystem object
S3_ENDPOINT_URL = "https://" + os.environ["AWS_S3_ENDPOINT"]
fs = s3fs.S3FileSystem(client_kwargs={'endpoint_url': S3_ENDPOINT_URL})
BUCKET = "radjerad/diffusion/code_postal"
FILE_KEY_S3 = "laposte_hexasmal.csv"
FILE_PATH_S3 = BUCKET + "/" + FILE_KEY_S3

with fs.open(FILE_PATH_S3, mode="rb") as file_in:
    code_postal = pd.read_csv(file_in, sep=";")

In [ ]:
code_postal.head()

In [ ]:
code_postal.shape # 39 193

In [ ]:
df_rep_pm.loc[df_rep_pm["code_postal"].isna(), :].shape # 528 795

In [ ]:
df_rep_pm.loc[df_rep_pm["code_commune"].isna(), :].shape # 1 441 393

In [ ]:
# donc plus de code commune manquants que de code postaux
# on va merger code postal et code_insee
# code_postal = code_postal.loc[:, ["Code_commune_INSEE", "Code_postal"]]

In [ ]:
code_postal.columns = [c.lower() for c in code_postal.columns]

In [ ]:
#df_rep_pm['code_postal2'] = pd.to_numeric(df_rep_pm['code_postal'], errors='coerce')

In [ ]:
# df_rep_pm.loc[df_rep_pm["code_postal2"].isna(),:].shape # 540862 , on a remplacé des valeurs bizarres en valeurs manquantes

In [ ]:
code_postal.head()

In [ ]:
code_postal["code_departement"] = code_postal["code_commune_insee"].str.slice(start=0,stop=2) # a corriger pour les drom

In [ ]:
code_postal[["code_departement", "code_commune_insee"]].head()

In [ ]:
df_rep_pm["code_departement"] = df_rep_pm["code_postal"].astype(str).str.slice(start=0, stop=2)

In [ ]:
df_rep_pm[["code_departement", "code_postal"]].head()

In [ ]:
code_postal["commune"] = code_postal["nom_commune"].str.lower()

In [ ]:
code_postal["commune"] = (code_postal["commune"].str.replace("à", "a")
                                            .str.replace("è", "e")
                                            .str.replace("é", "e")
                                            .str.replace("ç", "c")
                                            .str.replace("ù", "u")
                                            .str.replace("ô", "o"))

In [ ]:
code_postal["commune"] = (code_postal["commune"].str.replace("SAINT", "ST"))

In [ ]:
code_postal["commune"] = code_postal["commune"].str.replace(" ", "")

In [ ]:
code_postal["commune"].value_counts()

In [ ]:
code_postal.duplicated(subset=["commune", "code_departement"]).sum() # 4113

In [ ]:
code_postal.shape # 39193 -> 35080

In [ ]:
code_postal = code_postal.drop_duplicates(subset = ["commune", "code_departement"])

In [ ]:
code_postal.head(20)

In [ ]:
code_postal = code_postal.loc[:, ["code_commune_insee", "code_departement", "commune"]]

In [ ]:
df_rep_pm.shape # 9 751 833

In [ ]:
df_rep_pm = pd.merge(df_rep_pm, code_postal, on=["code_departement", "commune"], how="left")

In [ ]:
df_rep_pm.shape # 9751833

In [ ]:
import functions as fc
fc.export2minio("df_rep_pm_2017_commune.csv", df_rep_pm)

In [ ]:
df_rep_pm.loc[df_rep_pm["code_commune_insee"].isna(), :].sum()

In [ ]:
df_rep_pm.head()